# Scraping data from BoligPortal

In this notebook, we connect to the www.boligportal.dk and collect data on all available accomodations in the Greater Copenhagen area as of the day on which this notebook is executed.

In this notebook, we go through the following workflow:

1. We identify all web pages that show relevant search results, gather all the links to all accommodation listings and download their individual web pages.
2. The data is cleaned and put together in a nicely formatted dataframe where multiple kinds of information for the accommodation listings are made available for data visualization & analytics.

## User input for the analysis

In [1]:
"""
====================
Providing user input
====================
"""

# Website to be scraped
StartPage = "https://www.boligportal.dk/lejeboliger/k%C3%B8benhavn/"
WebSite = "https://www.boligportal.dk"

# Defining file storage location: either "Drive" or "Local"
FileStorageForUse = "Drive"

# If using Google Drive, specify project folder
ProjectFolder = "Projects/IT/BoligPortal insights/"

## Setting things up

We start out by importing all relevant packages for our work and by defining some custom functions that will help us extract the data that we need.

In [2]:
import datetime as dt
Today = dt.date.today()
print("This notebook was last updated on:", Today)
print("Author: Kiril Boyanov (kirilboyanovbg@gmail.com)")
print("Connect on LinkedIn: https://www.linkedin.com/in/kirilboyanov/")

This notebook was last updated on: 2022-10-19
Author: Kiril Boyanov (kirilboyanovbg@gmail.com)
Connect on LinkedIn: https://www.linkedin.com/in/kirilboyanov/


In [3]:
%%capture
# Installing packages that may not always be available
# This is mostly relevant for Google Colab
!pip install googlemaps
!pip install gmaps

In [4]:
"""
===========================
Importing relevant packages
===========================
"""

# For general and data-related work
import pandas as pd
import numpy as np
import math
import locale

# For general text-related work
from re import search
import json

# For scraping from the web
import requests
from bs4 import BeautifulSoup
import urllib
import lxml.html

# For geolocation purposes
import googlemaps
import gmaps

In [5]:
"""
=======================================
Arranging data storage for the analysis
=======================================
"""

# Creating a universal folder reference to use regardless of chosen storage method
if FileStorageForUse == "Local":
  import os
  AnalysisFolder = os.getcwd()
  print("The analysis will be run using local data storage.")
  print("The data will be saved in the following directory:")
  print(AnalysisFolder)
elif FileStorageForUse == "Drive":
  from google.colab import drive
  drive.mount('/content/gdrive/', force_remount = True)
  AnalysisFolder = "/content/gdrive/MyDrive/" + ProjectFolder
  print("The analysis will be run using Google Drive for data storage.")
  print("The data will be saved in the following directory:")
  print(AnalysisFolder)
else:
  print("Incorrect output, please check the '' input before proceeding.")

Mounted at /content/gdrive/
The analysis will be run using Google Drive for data storage.
The data will be saved in the following directory:
/content/gdrive/MyDrive/Projects/IT/BoligPortal insights/


In [ ]:
"""
==============================================================
Custom function to find substring between two other substrings
==============================================================
"""

def FindStringBetween(InputString, Sub1, Sub2):

  """
  Finds a substring located between two other substrings (sub1 and sub2).
  Returns the subtring.
  """
  # Getting index of substrings
  Index1 = InputString.find(Sub1)
  Index2 = InputString.find(Sub2)
  
  # Locating the string and returning it to the user
  Result = InputString[Index1 + len(Sub1) : Index2]
  return Result

In [ ]:
"""
===============================================================================
Custom function that extracts links to accommodations listed on a specific page
===============================================================================
"""

def ExtractAccommodationsOnPage(PageURL, PageNum):

  """
  Extracts links to accommodation entries listed on a specific page (PageURL).
  Combined with a list of page numbers/IDs, it marks the page from which the
  links were extracted as well as the date & time on which the extraction took
  place.
  """

  # Getting the contents of the page
  Page = requests.get(PageURL)

  # Parsing the HTML content
  Soup = BeautifulSoup(Page.content, "html.parser")

  # Initiating a list to store the result
  LinksOnPage = []

  # Finding all page links and adding them to the list
  for link in Soup.find_all("a"):
    data = link.get('href')
    LinksOnPage.append(data)

  # Creating a nicely formatted dataframe
  AccommodationsOnPage = pd.DataFrame({"Link":LinksOnPage})

  # Keeping only rows where the links are related to actual accommodation
  AccommodationsOnPage["LinkToAccommodation"] = AccommodationsOnPage["Link"].str.contains("-id-")
  AccommodationsOnPage = AccommodationsOnPage[AccommodationsOnPage["LinkToAccommodation"] == True].copy()
  AccommodationsOnPage.reset_index(inplace = True, drop = True)
  AccommodationsOnPage.drop(columns = ["LinkToAccommodation"], inplace = True)

  # Reparing the link so that it also includes the website
  AccommodationsOnPage["Link"] = WebSite + AccommodationsOnPage["Link"]
  AccommodationsOnPage["PageID"] = PageNum
  AccommodationsOnPage["Timestamp"] = dt.datetime.now()

  # Returning the data extracted to the user
  return AccommodationsOnPage

In [ ]:
"""
==========================================================================
Custom function to extract most likely accommodation address from web page
==========================================================================
"""

def ExtractLikelyAddress(Soup, DivClass = "css-1bbi9fj"):

  """
  Extracts the most likely address string from a specific BoligPortal web page.
  Requires that we've already generated a 'Soup' for the web page in question
  and that we specify the correct 'DivClass' that can help us find the address.
  """

  # Getting a list of entries, one of which should be the address
  PossibleAddressEntries = Soup.find_all("div", class_= DivClass)
  ListOfEntries = []
  ListOfLengths = []

  for entry in PossibleAddressEntries:
    string_entry = str(entry)
    string_length = len(string_entry)
    ListOfEntries.append(string_entry)
    ListOfLengths.append(string_length)

  # The address is likely the entry with the highest number of characters
  MaxValueIndex = ListOfLengths.index(max(ListOfLengths))
  MostLikelyAddress = ListOfEntries[MaxValueIndex]

  # Extracting the string containing the most likely address entry
  #Sub1 = '<div class="css-1bbi9fj">'
  Sub1 = '<div class="' + DivClass + '">'
  Sub2 = '</div>'
  MostLikelyAddress = FindStringBetween(MostLikelyAddress, Sub1, Sub2)
  
  # Returning the value to the user
  return MostLikelyAddress

In [ ]:
"""
==============================================
Custom function that extracts geolocation data
==============================================
"""

def GetGeolocationData(AddressString, Key):

  """
  Returns Latitude and longitude for a custom address provided as string
  (AddressString). Requires a usable authentication key (Key) in order to
  be able to connect to the Google Geocode API.
  """

  # Connecting to the Geocode API and collecting data
  gmaps = googlemaps.Client(key = Key)
  GeocodeResult = gmaps.geocode(AddressString)

  # Extracting information on Latitude and longitude
  try:
    Latitude = GeocodeResult[0]["geometry"]["location"]["lat"]
    Longitude = GeocodeResult[0]["geometry"]["location"]["lng"]
  except:
    Latitude = np.nan
    Longitude = np.nan

  return Latitude, Longitude

In [ ]:
"""
=================================================
Custom function to extract text-based description
=================================================
"""

def ExtractLikelyDescription(Soup, DivClass = "css-1f7mpex"):

  """
  Extracts what is likely to be the text-based description string
  listed on a specific BoligPortal web page.
  Requires that we've already generated a 'Soup' for the web page in question
  and that we specify the correct 'DivClass' that helps us find the description.
  """

  # Getting the string that represents the text-based description
  TextDescription = Soup.find_all("div", class_= DivClass)
  TextDescription = str(TextDescription)

  # Extracting the string containing the text description
  Sub1 = '<div class="' + DivClass + '">'
  Sub2 = '</div>'
  TextDescription = FindStringBetween(TextDescription, Sub1, Sub2)

  return TextDescription

In [ ]:
"""
================================================
Custom function to extract accommodation details
================================================
"""

def ExtractAccommodationDetails(Soup, SoupID, SpanClassForType = "css-1218edi", SpanClassForValue = "css-1e8e3fr"):

  """
  Extracts what details related to the accommodation, e.g. rent size, number
  of rooms, period for which the entry is being leased etc.
  Requires that we've already generated a 'Soup' for the web page in question
  and that we specify the correct 'SpanClass' for both the HTML component
  that contains the info type and the one that contains the actual value.
  """

  # Getting an HTML list of accommodation characteristics
  InfoTypes = Soup.find_all("span", class_= SpanClassForType)
  InfoTypes = str(InfoTypes)

  # Converting results to string and then to a Python list
  ReplacementString = '<span class="' + SpanClassForType + '">'
  InfoTypes = InfoTypes.replace(ReplacementString, "")
  InfoTypes = InfoTypes.replace('</span>', "")
  InfoTypes = InfoTypes.replace('[', "")
  InfoTypes = InfoTypes.replace(']', "")
  InfoTypes = InfoTypes.split(", ")

  # Getting an HTML list of values for those characteristics
  InfoValues = Soup.find_all("span", class_= SpanClassForValue)
  InfoValues = str(InfoValues)

  # Converting results to string and then to a Python list
  ReplacementString = '<span class="' + SpanClassForValue + '">'
  InfoValues = InfoValues.replace(ReplacementString, "")
  InfoValues = InfoValues.replace('</span>', "")
  InfoValues = InfoValues.replace('[', "")
  InfoValues = InfoValues.replace(']', "")
  InfoValues = InfoValues.split(", ")

  # Searching for information on energy rating
  EnergyRating = Soup.find_all("div", class_= "css-jalf26")

  # For entries that are hosting an "open house", we will have 2 additional
  # InfoValues because they use the same CSS formatting; those need to be removed
  if len(InfoTypes) + 2 == len(InfoValues):
    InfoValues = InfoValues[2:]

  # Extracting information on energy rating wherever available
  # Energy rating is stored separately for those cases in which it is available
  # The rating is stored as a picture, so we need to extract it from the file name
  if len(InfoTypes) != len(InfoValues):
    EnergyRating = str(EnergyRating)
    # Extracting file name of the image
    Sub1 = '[<div class="css-jalf26"><style data-emotion="css rdsunt">.css-rdsunt{width:24px;}</style><img aria-describedby="id-3" class="css-rdsunt" src="'
    Sub2 = '" tabindex="0"/>'
    EnergyRating = FindStringBetween(EnergyRating, Sub1, Sub2)
    # Inserting the energy rating in the generic Python list
    InfoValues.insert(12, EnergyRating)

  # Creating a dataframe with the accommodation characteristics
  TempDetails = pd.DataFrame({"InfoType":InfoTypes, "InfoValue":InfoValues})
  TempDetails["TempID"] = 1

  # Removing potential duplicate characteristics
  # (this does occur on some posts)
  TempDetails.drop_duplicates(subset = ["InfoType"], inplace = True)

  # Transposing the dataframe so that we get the characteristics as columns instead
  TempDetails = pd.pivot(TempDetails, index = "TempID", columns = "InfoType", values = "InfoValue")
  TempDetails["Link"] = SoupID

  return TempDetails

In [ ]:
"""
===========================================
Custom function to extract landlord details
===========================================
"""

def ExtractLandlordDetails(Soup, SoupID, DivClassGeneral = "css-ubwy5d", DivClassCreated = "css-a70nv0"):

  """
  Extracts what details related to the accommodation, e.g. rent size, number
  of rooms, period for which the entry is being leased etc.
  Requires that we've already generated a 'Soup' for the web page in question
  and that we specify the correct 'SpanClass' for both the HTML component
  that contains the info type and the one that contains the actual value.
  """

  # Getting the entries related to general landlord information
  LandlordDescription = Soup.find_all("div", class_= DivClassGeneral)
  LandlordDescription = str(LandlordDescription)

  # Getting the entry related to when the landlord created their profile
  LandlordSince = Soup.find_all("div", class_= DivClassCreated)
  LandlordSince = str(LandlordSince)

  # Putting the data from the two different sources together
  LandlordDescription = LandlordDescription + ", " + LandlordSince

  # Replacing irrelevant strings and trimming whitespace
  IrrelevantStrings = ['<div class="' + DivClassGeneral + '">', \
                      '<div class="' + DivClassCreated + '">', \
                      '</div>', '<!-- -->', '</div>]', \
                      '<!-- --> </div>', '[', ']']

  for string in IrrelevantStrings:
    LandlordDescription = LandlordDescription.replace(string, "")

  # Splitting the information into a list and turning it into a dataframe
  LandlordDescription = LandlordDescription.split(", ")
  LandlordDescription = pd.DataFrame({"InfoType":LandlordDescription})

  # Removing potential whitespaces and duplicate entries
  LandlordDescription["InfoType"] = LandlordDescription["InfoType"].str.strip()
  LandlordDescription.drop_duplicates(subset = ["InfoType"], inplace = True)

  # Extracting data on validation
  LandlordDescription["LandlordValidated"] = np.max(LandlordDescription["InfoType"].str.contains("Valideret") == True)

  # Extracting data on number of postings
  LandlordDescription["LandlordNumberOfPosts"] = np.where(LandlordDescription["InfoType"].str.contains("annonc") == True, LandlordDescription["InfoType"], np.nan)
  LandlordDescription["LandlordNumberOfPosts"] = LandlordDescription["LandlordNumberOfPosts"].str.extract('(\d+)')
  LandlordDescription["LandlordNumberOfPosts"] = pd.to_numeric(LandlordDescription["LandlordNumberOfPosts"])
  LandlordDescription["LandlordNumberOfPosts"] = np.max(LandlordDescription["LandlordNumberOfPosts"])

  # Extracting data on most recent activity
  LandlordDescription["LandlordLastActive"] = np.where(LandlordDescription["InfoType"].str.contains("aktiv") == True, LandlordDescription["InfoType"], np.nan)
  LandlordDescription["LandlordLastActive"] = LandlordDescription["LandlordLastActive"].ffill()
  LandlordDescription["LandlordLastActive"] = LandlordDescription["LandlordLastActive"].bfill()
  LandlordDescription["LandlordLastActive"] = LandlordDescription["LandlordLastActive"].str.replace("Sidst aktiv: ", "", regex = False)

  # Extracting data on when the landlord registered with BoligPortal
  LandlordDescription["LandlordSince"] = np.where(LandlordDescription["InfoType"].str.contains("Oprettet") == True, LandlordDescription["InfoType"], np.nan)
  LandlordDescription["LandlordSince"] = LandlordDescription["LandlordSince"].ffill()
  LandlordDescription["LandlordSince"] = LandlordDescription["LandlordSince"].bfill()
  LandlordDescription["LandlordSince"] = LandlordDescription["LandlordSince"].str.replace("Oprettet: ", "", regex = False)

  # Adding an ID associated with the post and keeping only 1 row of data
  LandlordDescription["Link"] = SoupID
  LandlordDescription.drop_duplicates(subset = ["Link"], inplace = True)

  # Returning a one-row table to the user
  return LandlordDescription

## Collecting links to all pages showing accommodation entries

The next step is to connect to BoligPortal and collect all relevant links that point to web pages where single accommodation entries are listed.

To be able to do that, we first need to see how many pages with search results there are and generate links to those. Following this, we extract all relevant links from all pages showing search results.

In [ ]:
"""
=================================================
Extracting number of available pages with results
=================================================
"""

# Downloading and parsing the HTML content
Page = requests.get(StartPage)
Soup = BeautifulSoup(Page.content, "html.parser")

# Locating buttons from the bottom of the page that contain page numbers
ButtonElements = Soup.find_all("button", class_ ="temporaryButtonClassname css-1qlr7c3")

# Specifying HTML code enclosing the page numbers
before_string = '<button class="temporaryButtonClassname css-1qlr7c3" type="button"><span class="css-176v3d">'
after_string = '</span>'

# Extracting page numbers strings from those buttons
PageStrings = []
for element in ButtonElements:
  temp_element = str(element)
  temp_string = FindStringBetween(temp_element, before_string, after_string)
  PageStrings.append(temp_string)

# Converting the page number strings into actual numbers
PageNumbers = []

for num in PageStrings:
  try:
    num_float = int(num)
    PageNumbers.append(num_float)
  except:
    pass

# Keeping the maximum number only (reveals what the last page is)
NumberOfPages = max(PageNumbers)

# Printing a confirmation to the user
print("At the time of reaching the website, the maximum number of pages with accommodation listings was", NumberOfPages, ".")

At the time of reaching the website, the maximum number of pages with accommodation listings was 160 .


In [ ]:
"""
==========================================================
Creating list of links to all available pages with results
==========================================================
"""

# Results look like this, with 18 being displayed on each page:
# https://www.boligportal.dk/lejeboliger/k%C3%B8benhavn/?offset=0

# Calculating the maximum offset corresponding to the last page in the search results
MaxOffsets = (NumberOfPages - 1)*18
MaxOffsets = math.ceil(MaxOffsets)
MaxOffsets

# Creating a list of offsets, with 18 offsets representing a new page
Offsets = 0
PageOffsets = []

while Offsets <= MaxOffsets:
  PageOffsets.append(Offsets)
  Offsets = Offsets + 18

# Creating a list of links to all pages containing search results
BasicLinkString = "https://www.boligportal.dk/lejeboliger/k%C3%B8benhavn/?offset="
ResultPageLinks = []

for entry in PageOffsets:
  PageLink = BasicLinkString + str(entry)
  ResultPageLinks.append(PageLink)

# Creating a list of simple page numbers
ResultPageNumbers = np.arange(1, len(ResultPageLinks) + 1)

In [ ]:
"""
==============================================================================
Extracting links to accommodation entries from all pages in the search results
==============================================================================
"""

# Dataframe to store all extracted links
AccommodationLinks = pd.DataFrame()

# Extracting the links using a loop
for page_link, page_num in zip(ResultPageLinks, ResultPageNumbers):
  TempData = ExtractAccommodationsOnPage(page_link, page_num)
  AccommodationLinks = pd.concat([AccommodationLinks, TempData], ignore_index = True)

A preview of what the data (links) collected looks like is shown below:

In [ ]:
# Getting a preview of the links collected
AccommodationLinks

,Link,PageID,Timestamp
0,https://www.boligportal.dk/v%C3%A6relser/k%C3%...,1,2022-10-11 06:19:22.792957
1,https://www.boligportal.dk/lejligheder/k%C3%B8...,1,2022-10-11 06:19:22.792957
2,https://www.boligportal.dk/lejligheder/k%C3%B8...,1,2022-10-11 06:19:22.792957
3,https://www.boligportal.dk/lejligheder/k%C3%B8...,1,2022-10-11 06:19:22.792957
4,https://www.boligportal.dk/lejligheder/k%C3%B8...,1,2022-10-11 06:19:22.792957
...,...,...,...
2858,https://www.boligportal.dk/v%C3%A6relser/k%C3%...,159,2022-10-11 06:21:53.336746
2859,https://www.boligportal.dk/v%C3%A6relser/k%C3%...,159,2022-10-11 06:21:53.336746
2860,https://www.boligportal.dk/lejligheder/k%C3%B8...,159,2022-10-11 06:21:53.336746
2861,https://www.boligportal.dk/v%C3%A6relser/k%C3%...,159,2022-10-11 06:21:53.336746


## Downloading content for all relevant pages with accommodation listings

### Downloading the data

Below, we use all the links from the above table and download the content of the web pages associated with the single accommodation listings.

In [ ]:
"""
===============================================
Downloading content from all relevant web pages
===============================================
"""

# Initiating lists to store the results
AllSoups = []
AllSoupIDs = []

# Gathering information from all pages while allowing for pages that may have
# been deleted since obtaining the list of links not to disrupt the workflow
for link in AccommodationLinks["Link"]:
  try:
    Page = requests.get(link)
    Soup = BeautifulSoup(Page.content, "html.parser")
    AllSoups.append(Soup)
    AllSoupIDs.append(link)
  except:
    pass

# Printing a confirmation to the end user
print("Downloading of content from", len(AllSoups), "web pages complete.")

### Exporting the data

These web pages are kept as `BeautifulSoup` objects and are then exported to a local file so that they can be used in the `BoligPortal data scraping - format.ipynb` notebook, where the web-based data is cleaned up and formatted in a way that makes is useful for analytical purposes.

In [ ]:
# Creating a dataframe
BeautifulSoupData = pd.DataFrame({"SoupObject":AllSoups,"SoupLink":AllSoupIDs})

# Exporting the data to a CSV file so that the data can be re-imported in case we disconnect
BeautifulSoupData.to_csv(AnalysisFolder + "Data/BeautifulSoupData.csv", index = False)

# Printing a confirmation to the end user
print("The data was been temporarily exported to the local drive on:", dt.datetime.now().strftime("%d %b %Y, %H:%M"))

The data was been temporarily exported to the local drive on: 11 Oct 2022, 07:38


## Extracting data for individual accommodations from their own pages

The following bits of information need to be extracted:

* ID of the accommodation item *(done as of 11-10-2022)*
* Address *(done as of 11-10-2022)*
* Text-based description *(done as of 11-10-2022)*
* Details on the accommodation and the lease *(WIP focused on formatting as of 12-10-2022)*
* Details on the landlord *(WIP focused on formatting as of 12-10-2022)*

After the information is extracted, it's put together in a nicely formatted dataframe, which can then be used for data visualization & analytics.

### Creating a dataframe to store the output

We start out by creating a dataframe to hold the output with all the nicely cleaned up data.

In [ ]:
# Initiating a dataframe to store the results
Accommodations = AccommodationLinks.copy()

### Extracting the ID of the accommodation

On BoligPortal, each accommodation has its own ID, which is also obvious from the URL address of the pages containing actual accommodation listings. To serve as an example, the first two URLs are shown below:

In [ ]:
# Printing the two top entries in the data
for entry in Accommodations["Link"][0:3]:
  print(entry)

https://www.boligportal.dk/v%C3%A6relser/k%C3%B8benhavn/14m2-1-vaer-id-5240420
https://www.boligportal.dk/lejligheder/k%C3%B8benhavn/56m2-2-vaer-id-3887217
https://www.boligportal.dk/lejligheder/k%C3%B8benhavn/240m2-5-vaer-id-5308773


The ID is extracted from the URL in a fairly straighforward manner and is then attached to the dataframe.

In [ ]:
# Extracting information on the ID associated with each entry
Accommodations["StringIndex"] = Accommodations["Link"].str.find("-id-") + 4
Accommodations["StringLength"] = Accommodations["Link"].str.len()
Accommodations["EntryID"] = Accommodations.apply(lambda x: x.Link[x.StringIndex:x.StringLength], axis = 1)

# Removing superfluous columns
Accommodations.drop(columns = ["StringIndex", "StringLength"], inplace = True)

In [ ]:
# Temporary export of the data (to be disabled when the notebook is fully operational)
# Accommodations.to_pickle(AnalysisFolder + "Data/Accommodations_IDs.pkl")
# Accommodations.to_excel(AnalysisFolder + "Data/Accommodations_IDs.xlsx", index = True)

In [ ]:
# Temporary import of the data (to be disabled when the notebook is fully operational)
# Accommodations = pd.read_pickle(AnalysisFolder + "Data/Accommodations_IDs.pkl")

### Extracting data on physical location

#### Pre-requisite: getting post numbers for Denmark

Before we continue with creating address data, we need to download a list of all post codes in Denmark, which is freely available on [PostNord's website](https://www.postnord.dk/kundeservice/kundeservice-erhverv/om-postnumre/postnummerkort-postnummerfiler). For being on the safe side, a copy of this file is also downloaded to the `Data` folder in this project.

A preview of what that list looks like is shown below:

In [ ]:
# Importing data on postcodes from PostNord and adding town name based on it
PostCodes = pd.read_excel(AnalysisFolder + "Data/Post numbers in Denmark.xlsx", skiprows = 1, converters={'Postnr.':str})
PostCodes.columns = ["PostCode", "TownOrCity", "Street", "Company", "Province", "Country"]

# Keeping relevant columns only
PostCodes = PostCodes[["PostCode", "TownOrCity"]].copy()

# Removing any potential duplicates
PostCodes.drop_duplicates(subset = ["PostCode"], inplace = True)
PostCodes.reset_index(inplace = True, drop = True)

# Previewing the data
PostCodes[550:555]

,PostCode,TownOrCity
550,1953,Frederiksberg C
551,1954,Frederiksberg C
552,1955,Frederiksberg C
553,1956,Frederiksberg C
554,1957,Frederiksberg C


#### Adding information on location

First, we extract address-related data and then we use geolocation to put the address on the map.

##### Extracting and cleaning up address data

Below, we use the address string from the web page to extract the following kinds of information:

* Street name
* Post code
* Town or city name
* Municipality name

In [ ]:
# Extracting addresses for all accommodation entries linked
ListOfAddresses = []

for soup in AllSoups:
  address_string = ExtractLikelyAddress(soup, "css-1bbi9fj")
  ListOfAddresses.append(address_string)

# Creating a small dataframe with address data only
Info_Addresses = pd.DataFrame({"Link":AllSoupIDs, "PhysicalAddressDetailed":ListOfAddresses})

# Extracting road name and postcode
Info_Addresses["StringIndex"] = Info_Addresses["PhysicalAddressDetailed"].str.find(", ")
Info_Addresses["StreetName"] = Info_Addresses.apply(lambda x: x.PhysicalAddressDetailed[0:x.StringIndex], axis = 1)
Info_Addresses["PostCode"] = Info_Addresses.apply(lambda x: x.PhysicalAddressDetailed[(x.StringIndex + 2):(x.StringIndex + 6)], axis = 1)

# Adding data on town/city based on post code
Info_Addresses = pd.merge(Info_Addresses, PostCodes, how = "left", on = "PostCode")

# Creating a clean address that does not have information on the floor
Info_Addresses["PhysicalAddress"] = Info_Addresses["StreetName"] + ", " + Info_Addresses["PostCode"] + " " + Info_Addresses["TownOrCity"]

# Extracting information reg. the floor on which the accommodation is located
Info_Addresses["StringCount"] = Info_Addresses["PhysicalAddressDetailed"].str.contains(" - ")
Info_Addresses["StringIndex"] = np.where(Info_Addresses["StringCount"] == True, Info_Addresses["PhysicalAddressDetailed"].str.find(" - "), np.nan)
Info_Addresses["StringLength"] = Info_Addresses["PhysicalAddressDetailed"].str.len()
Info_Addresses["StringIndex"].fillna(Info_Addresses["StringLength"], inplace = True)
Info_Addresses["StringIndex"] = Info_Addresses["StringIndex"].astype("int32")
Info_Addresses["FloorText"] = np.where(Info_Addresses["StringCount"] == True, Info_Addresses.apply(lambda x: x.PhysicalAddressDetailed[(x.StringIndex + 3):x.StringLength], axis = 1), np.nan)

# Repairing the formatting of the "Floor" variable and converting it to a numeric one
Info_Addresses["Floor"] = Info_Addresses["FloorText"].str.replace(". sal", "", regex = False)
Info_Addresses["Floor"] = Info_Addresses["Floor"].str.replace("Stuen", "0", regex = False)
Info_Addresses["Floor"] = Info_Addresses["Floor"].str.replace("Kælder", "-1", regex = False)
Info_Addresses["Floor"] = Info_Addresses["Floor"].astype("float")

# Removing superfluous columns
Info_Addresses.drop(columns = ["PhysicalAddressDetailed", "StringIndex", "StringCount", "StringLength"], inplace = True)

In order to improve the quality of the data even more, we go further by manually grouping the different towns/cities into their respective municipalities.

This is done manually through a spreadsheet we create and maintain in the `Data` folder of this analysis. A preview of the data is shown below:

In [ ]:
# Use the line below to get entries to update the spreadsheet
# (Can be relevant if new entries appear on the list, though that is unlikely)
# Info_Addresses.TownOrCity.unique()

In [ ]:
# Importing manual mapping of municipalities
Municipalities = pd.read_excel(AnalysisFolder + "Data/Municipalities.xlsx")

# Applying the manual mapping
Info_Addresses = pd.merge(Info_Addresses, Municipalities, how = "left", on = "TownOrCity")

# Previewing the data
Municipalities.head(5)

,TownOrCity,Municipality
0,Albertslund,Albertslund
1,Bagsværd,Gladsaxe
2,Brøndby Strand,Brøndby
3,Brønshøj,København
4,Charlottenlund,Gentofte


##### Adding geolocation data

Below, we use the `PhysicalAddress` column to estimate the geographic coordinates (Latitude and longitude) for the various addresses.

In [ ]:
# Temporary export of the data (to be disabled when the notebook is fully operational)
# Info_Addresses.to_pickle(AnalysisFolder + "Data/Info_Addresses.pkl")
# Info_Addresses.to_excel(AnalysisFolder + "Data/Info_Addresses.xlsx", index = True)

In [ ]:
# Temporary import of the data (to be disabled when the notebook is fully operational)
# Info_Addresses = pd.read_pickle(AnalysisFolder + "Data/Info_Addresses.pkl")

In [ ]:
# Adding address-related data to the rest of the data
Accommodations = pd.merge(Accommodations, Info_Addresses, how = "left", on = "Link")

#### Adding geolocation data

Geolocation (estimation of Latitude and longitude) is done by using Google's API, access to which is arranged and managed on [this website](https://console.cloud.google.com/google/maps-apis/overview). Before running the notebook, you need to ensure that you have enabled this API and that you have got a corresponding `AUTH_KEY` that you can use.

This key is imported from a local file in here (the file is not available online).

In [ ]:
# You need an API key from Google (AUTH_KEY) in order to use this functionality
# https://developers.google.com/maps/documentation/geocoding/intro

# Importing the authentication key from a locally stored TXT file
with open(AnalysisFolder + "/Data/AUTH_KEY.txt") as file:
    AUTH_KEY = file.read()
    file.close()

In [ ]:
# Keeping only unique addresses and adding country name
# This helps reduce the number of requests sent to the Google API
# and it therefore makes it "cheaper" to run the analysis
UniqueAddresses = Accommodations[["PhysicalAddress"]].copy()
UniqueAddresses["AddressWithCountry"] = UniqueAddresses["PhysicalAddress"] + ", Denmark"
UniqueAddresses.drop_duplicates(subset = ["PhysicalAddress"], inplace = True)

# Initializing lists to store the geolocation data
ListOfLat = []
ListOfLng = []

# Using a loop to generate the data
for address in UniqueAddresses["AddressWithCountry"]:
  TempGeoData = GetGeolocationData(address, AUTH_KEY)
  TempLatitude = TempGeoData[0]
  TempLongitude = TempGeoData[1]
  ListOfLat.append(TempLatitude)
  ListOfLng.append(TempLongitude)

# Adding the data to the dataframe and removing superfluous columns
UniqueAddresses["Latitude"] = ListOfLat
UniqueAddresses["Longitude"] = ListOfLng
UniqueAddresses.drop(columns = ["AddressWithCountry"], inplace = True)

In [ ]:
# Temporary export of the data (to be disabled when the notebook is fully operational)
# UniqueAddresses.to_pickle(AnalysisFolder + "Data/UniqueAddresses.pkl")
# UniqueAddresses.to_excel(AnalysisFolder + "Data/UniqueAddresses.xlsx", index = True)

In [ ]:
# Temporary import of the data (to be disabled when the notebook is fully operational)
# UniqueAddresses = pd.read_pickle(AnalysisFolder + "Data/UniqueAddresses.pkl")

In [ ]:
# Merging the data back into the main dataframe
Accommodations = pd.merge(Accommodations, UniqueAddresses, how = "left", on = "PhysicalAddress")

# Calculating stats on how successful the geolocation was
N_Rows = len(Accommodations)
N_Missing = Accommodations["Latitude"].isna().sum()
N_Located = Accommodations["Latitude"].notna().sum()
Pct_Missing = round((N_Missing/N_Rows)*100, 2)
Pct_Located = round((N_Located/N_Rows)*100, 2)

# Printing out a confirmation to the user
print("Geolocation data has been successfully sourced for", N_Located, "out of", N_Rows, "addresses (", Pct_Located, "%).")

Geolocation data has been successfully sourced for 2861 out of 2863 addresses ( 99.93 %).


### Extracting text-based descriptions

Below, we extract the text description that is provided for (most) accommodations listed on the BoligPortal website. This information can later be used for text analytics. A preview of the first three text-based descriptions is shown below:

In [ ]:
# Extracting text-based descriptions for all accommodation entries linked
ListOfDescriptions = []

for soup in AllSoups:
  desc_string = ExtractLikelyDescription(soup, "css-1f7mpex")
  ListOfDescriptions.append(desc_string)

# Previewing the first two descriptions
for desc in ListOfDescriptions[0:3]:
  print(desc, "\n")

Cozy room in a shared flat for rent (Room 1).
The room is on ground in a nice house in a good neighbourhood. 
Furniture is included as well as all supplies (electricity, heat, water and internet). Wifi from fiber.
Washing facilities are also included along with a fitness room. 
The house is very close to Emdrup Station. From here you can get to the city center in 15 minutes ny train

The kitchen and bath is shared with 2 other tenants 

God 2 værelses lejlighed med stor stue. Stort soveværelse med stort skab. Godt køkken. Der er en god gård og lejligheden ligger tæt på Spinderiet og Valby station. 

Lejligheden er i 2 plan øverst på 4. og 5. sal med elevator.
Lejligheden er uformet med et usædvanligt smukt køkken alrum og udgang til stor terrasse. Der er Dinesen gulve overalt.
Dere r 2 stuer samt 3 soveværelser og 2 badeværelser, det ene med badekar.
Lejligheden er meget original og eksklusiv. 
Lejligheden ligger på Sankt Annæ Plads i en smuk gård. 



In [ ]:
# Temporary export of the data (to be disabled when the notebook is fully operational)
# ListOfDescriptions = pd.DataFrame({"List":ListOfDescriptions})
# ListOfDescriptions.to_pickle(AnalysisFolder + "Data/ListOfDescriptions.pkl")

In [ ]:
# Temporary export of the data (to be disabled when the notebook is fully operational)
# ListOfDescriptions = pd.read_pickle(AnalysisFolder + "Data/ListOfDescriptions.pkl")
# ListOfDescriptions = ListOfDescriptions["List"].to_list()

In [ ]:
# Adding the data to our dataframe
Accommodations["Description"] = ListOfDescriptions

### Extracting details related to the accommodation and the lease

Below, we extract different bits of information related to both the accommodation entry itself (e.g. number of rooms, whether pets are allowed etc.) as well as to the nature of the lease (e.g. how much the rent is, how much the deposit costs etc.). The data is then added to our main dataframe.

#### Extracting the data

In [ ]:
# Creating a dataframe to store the characteristics
AccommodationDetails = pd.DataFrame()

# Extracting accommodation characteristics
for soup, id in zip(AllSoups, AllSoupIDs):
  #print(id) # can be un-commented for troubleshooting purposes
  TempChar = ExtractAccommodationDetails(soup, id, SpanClassForType = "css-1218edi", SpanClassForValue = "css-1e8e3fr")
  AccommodationDetails = pd.concat([AccommodationDetails, TempChar], ignore_index = True)

# Cleaning up in the dataframe and previewing the data
AccommodationDetails.reset_index(inplace = True, drop = True)

Unfortunately, the data comes in a format that makes it difficult to use as the preview below indicates. Particularly, we need to have sensible column names, columns that represent numbers or dates need to be formatted as such etc.:


In [ ]:
# Previewing the data
AccommodationDetails.head(5)

InfoType,Aconto,Altan/terrasse,Boligtype,Delevenlig,Depositum,Elevator,Energimærke,Etage,Forudbetalt husleje,Husdyr tilladt,...,Møbleret tagterrasse,Vedligeholdelse,Vaskefaciliteter,Cykelskur,Fjernvarme,Fællesareal,Kælderrum,Terrasse/Have,Induktionskogeplade,Køle-/fryseskab
0,0 kr.,Nej,Værelse,Ja,5.800 kr.,Nej,/static/images/energy_labels/C_str2.png,Stuen,5.800 kr.,Nej,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,911 kr.,Nej,Lejlighed,Nej,31.500 kr.,Nej,-,Stuen,10.500 kr.,Nej,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.000 kr.,Nej,Lejlighed,Nej,150.000 kr.,Nej,-,4.,NaN,Ja,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.000 kr.,Nej,Lejlighed,Nej,27.000 kr.,Ja,-,21.,27.000 kr.,Ja,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.000 kr.,Nej,Lejlighed,Nej,27.000 kr.,Ja,-,22.,27.000 kr.,Nej,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Formatting the data

We perform the following kind of data corrections:

* Columns containin currency data (like `Acconto` or `MonthlyRent`) get converted to numeric and the currency string ("kr.") gets stripped away
* Columns that should be formatted as numbers but which may contain some strings in them (like `Floor`) are converted into numeric, where the strings are represented by meaningful numbers (e.g. a ground floor will be represented as a "0")
* Columns that contain dates are converted to Python's native `datetime` format
* Columns that contain "Yes/No" data ("Ja/Nej" in Danish) are converted to Python's native `boolean` format

In [ ]:
AccommodationDetails.head(5)

InfoType,Aconto,Altan/terrasse,Boligtype,Delevenlig,Depositum,Elevator,Energimærke,Etage,Forudbetalt husleje,Husdyr tilladt,...,Møbleret tagterrasse,Vedligeholdelse,Vaskefaciliteter,Cykelskur,Fjernvarme,Fællesareal,Kælderrum,Terrasse/Have,Induktionskogeplade,Køle-/fryseskab
0,0 kr.,Nej,Værelse,Ja,5.800 kr.,Nej,/static/images/energy_labels/C_str2.png,Stuen,5.800 kr.,Nej,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,911 kr.,Nej,Lejlighed,Nej,31.500 kr.,Nej,-,Stuen,10.500 kr.,Nej,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.000 kr.,Nej,Lejlighed,Nej,150.000 kr.,Nej,-,4.,NaN,Ja,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.000 kr.,Nej,Lejlighed,Nej,27.000 kr.,Ja,-,21.,27.000 kr.,Ja,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.000 kr.,Nej,Lejlighed,Nej,27.000 kr.,Ja,-,22.,27.000 kr.,Nej,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Importing custom mapping table for column names
DetailsColNames = pd.read_excel(AnalysisFolder + "Data/Accommodation details column names.xlsx")

# Automatically renaming columns based on our mapping table
# An error is raised if new/unknown values are encoutered
try:
  AccommodationDetails.rename(columns = dict(zip(DetailsColNames["ColumnNameWeb"], DetailsColNames["ColumnNameProper"])), inplace = True)
except:
  print("New or unknown values are available in the data. Please review and update the mapping table in the 'Accommodation details column names.xlsx' spreadsheet.")

In [ ]:
# Repairing the formatting of columns with currency data
RelevantCols = ["Aconto", "Deposit", "Deposit2", "PrepaidRent", \
                "MovingInPrice", "MonthlyRent", "NumberOfBathrooms", \
                "NumberOfToiletsNoShower"]

for col in RelevantCols:
  AccommodationDetails[col] = AccommodationDetails[col].str.replace(".", "", regex = False)
  AccommodationDetails[col] = AccommodationDetails[col].str.extract('(\d+)')
  AccommodationDetails[col] = AccommodationDetails[col].astype(float)

# Sometimes, we have missing data in the first "Deposit" column
# In those cases, we attempt to copy the info from the "Deposit2" column
AccommodationDetails["Deposit"] = np.where(AccommodationDetails["Deposit"].notna(), AccommodationDetails["Deposit"], AccommodationDetails["Deposit2"])
AccommodationDetails.drop(columns = ["Deposit2"], inplace = True)

In [ ]:
"""
========================================================================
Repairing the formatting of columns that should be numeric but aren't it
========================================================================
In here, corrections are made separately for each variable as the nature of the
corrections that need to be performed depends on the nature of the data
"""

# Floor
AccommodationDetails["Floor"] = np.where(AccommodationDetails["Floor"] == "-", np.nan, AccommodationDetails["Floor"])
AccommodationDetails["Floor"] = AccommodationDetails["Floor"].str.replace("Stuen", "0", regex = False)
AccommodationDetails["Floor"] = AccommodationDetails["Floor"].str.replace("Kælder", "-1", regex = False)
AccommodationDetails["Floor"] = AccommodationDetails["Floor"].astype(float)

# SizeSquareMeters
AccommodationDetails["SizeSquareMeters"] = AccommodationDetails["SizeSquareMeters"].str.replace(" m²", "", regex = False)
AccommodationDetails["SizeSquareMeters"] = AccommodationDetails["SizeSquareMeters"].astype(float)

# NumberOfRooms
AccommodationDetails["NumberOfRooms"] = AccommodationDetails["NumberOfRooms"].astype(float)

In [ ]:
"""
========================================
Repairing the formatting of date columns
========================================
For accommodations that are available from as soon as possible, we impute
a day that is one week later than the date on which the accommodation was
published to the website (we assume you always need some time to take care
of the formalities related to seeing the place, signing a contract, etc.).
"""

# Specifying translation of Danish month names to numbers
MonthsInDanish = ["januar", "februar", "marts", "april", "maj", "juni", \
                  "juli", "august", "september", "oktober", "november",
                  "december"]
MonthsNumbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]

for month_dk, month_num in zip(MonthsInDanish, MonthsNumbers):
  AccommodationDetails["AvailableFrom"] = AccommodationDetails["AvailableFrom"].str.replace(month_dk, month_num, regex = False)

# AvailableFrom
AccommodationDetails["AvailableFromASAP"] = (AccommodationDetails["AvailableFrom"] == "Snarest muligt")
AccommodationDetails["AvailableFrom"] = np.where(AccommodationDetails["AvailableFromASAP"] == True, np.nan, AccommodationDetails["AvailableFrom"])
AccommodationDetails["AvailableFrom"] = pd.to_datetime(AccommodationDetails["AvailableFrom"], format = "%d. %m %Y", errors = "coerce")

# DateCreated
AccommodationDetails["DateCreated"] = pd.to_datetime(AccommodationDetails["DateCreated"], format = "%d.%m.%Y", errors = "coerce")

In [ ]:
"""
===========================================================
Translating values from Danish to English in string columns
===========================================================
and converting them to the a more usable data type (in some cases,
we need to keep the columns as strings or at least as categories)
"""

# Replacing values in "Yes/No" columns
RelevantCols = ["BalconyOrTerrace", "SuitableForSharing", "Elevator", \
                "PetsAllowed", "StudentsOnly", "Furnished", "Parking", \
                "SuitableForElderly", "Balcony", "CommonCourtyard", \
                "PlaygroundInCourtyard", "StorageRoom", "Dishwasher", \
                "RooftopTerrace", "Terrace", "Dryer", "WashingMachine", \
                "CommonRooftopTerrace", "ParkingAvailability", "OutdoorSpace", \
                "GardenOrCourtyard", "CableTV", "PrivateStorage", "Smoking", \
                "Terrace2", "Dryer2", "Fireplace", "GuestToilet", "Freezer", \
                "Shed", "Garden", "KitchenFan", "InternetIncluded", \
                "CeramicStoveAndBuiltInOven", "Refridgerator", "BuiltInOven", \
                "DoorbellAtEntrance", "WiFi_200Mbps", "FitnessCenter247", \
                "SecurityAvailable", "OwnWashingMachine", "EventsOnAnAnnualBasis", \
                "CozyCommonAreas", "FurnishedRooftopTerrace", "Maintenance", \
                "WashingFacilities", "BikeShed", "DistrictHeating", \
                "CommonAreas", "BasementRoom", "TerraceOrGarden", \
                "InductionStove", "FridgeAndFreezer"]

for col in RelevantCols:
  AccommodationDetails[col] = AccommodationDetails[col].str.replace("Ja", "1", regex = False)
  AccommodationDetails[col] = AccommodationDetails[col].str.replace("Fælles tagterrasse", "1", regex = False)
  AccommodationDetails[col] = AccommodationDetails[col].str.replace("Nej", "0", regex = False)
  AccommodationDetails[col] = AccommodationDetails[col].str.replace("-", "", regex = False)
  AccommodationDetails[col] = AccommodationDetails[col].astype(float)

In [ ]:
# Temporary preview - for testing purposes
AccommodationDetails.head(5)

InfoType,Aconto,BalconyOrTerrace,AccommodationType,SuitableForSharing,Deposit,Elevator,EnergyRating,Floor,PrepaidRent,PetsAllowed,...,Maintenance,WashingFacilities,BikeShed,DistrictHeating,CommonAreas,BasementRoom,TerraceOrGarden,InductionStove,FridgeAndFreezer,AvailableFromASAP
0,0.0,0.0,Værelse,1.0,5800.0,0.0,/static/images/energy_labels/C_str2.png,0.0,5800.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,911.0,0.0,Lejlighed,0.0,31500.0,0.0,-,0.0,10500.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2000.0,0.0,Lejlighed,0.0,150000.0,0.0,-,4.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,1000.0,0.0,Lejlighed,0.0,27000.0,1.0,-,21.0,27000.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,1000.0,0.0,Lejlighed,0.0,27000.0,1.0,-,22.0,27000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


The corrections of columns with custom formatting may become outdated if we start seeing new kinds of values in the source data. In such case, the lists of `OldValues` and `NewValues` that we're using below may need to be updated.

In [ ]:
"""
===========================================================================
Repairing columns that contain categorical variables with custom formatting
===========================================================================
Note: changes in the source data may necessitate changes in the values in the
lists that we use for the string substitutions below.
"""

# =================
# AccommodationType
# =================
OldValues = ["Værelse", "Lejlighed", "Hus", "Rækkehus"]
NewValues = ["Room", "Apartment", "House", "Semi-detached house"]

for old_val, new_val in zip(OldValues, NewValues):
  AccommodationDetails["AccommodationType"] = AccommodationDetails["AccommodationType"].str.replace(old_val, new_val, regex = False)

# EnergyRating
OldValues = ["/static/images/energy_labels/C_str2.png", \
             "-", \
             "/static/images/energy_labels/D_str2.png", \
             "/static/images/energy_labels/A20_str2.png", \
             "/static/images/energy_labels/A15_str2.png", \
             "/static/images/energy_labels/B_str2.png", \
             "/static/images/energy_labels/F_str2.png", \
             "/static/images/energy_labels/A10_str2.png", \
             "/static/images/energy_labels/E_str2.png", \
             "/static/images/energy_labels/G_str2.png"]
NewValues = ["C", "", "D", "A20", "A15", "B", "F", "A10", "E", "G"]

for old_val, new_val in zip(OldValues, NewValues):
  AccommodationDetails["EnergyRating"] = AccommodationDetails["EnergyRating"].str.replace(old_val, new_val, regex = False)

AccommodationDetails["EnergyRating"] = np.where(AccommodationDetails["EnergyRating"] == "", np.nan, AccommodationDetails["EnergyRating"])

# This variable is also converted to an ordered factor (categorical variable)
AccommodationDetails["EnergyRating"] = AccommodationDetails["EnergyRating"].astype("category")
AccommodationDetails["EnergyRating"] = AccommodationDetails["EnergyRating"].cat.reorder_categories(["G", "F", "E", "D", "C", "B", "A20", "A15", "A10"])

# ============
# RentalPeriod
# ============
OldValues = ["Ubegrænset", "12-23 måneder", "24+ måneder", "1-11 måneder"]
NewValues = ["Unlimited", "12-23 months", "24+ months", "1-11 months"]

for old_val, new_val in zip(OldValues, NewValues):
  AccommodationDetails["RentalPeriod"] = AccommodationDetails["RentalPeriod"].str.replace(old_val, new_val, regex = False)

# This variable is also converted to an ordered factor (categorical variable)
AccommodationDetails["RentalPeriod"] = AccommodationDetails["RentalPeriod"].astype("category")
AccommodationDetails["RentalPeriod"] = AccommodationDetails["RentalPeriod"].cat.reorder_categories(["1-11 months", "12-23 months", "24+ months", "Unlimited"])

In [ ]:
# Temporary export of the data (to be disabled when the notebook is fully operational)
# AccommodationDetails.to_pickle(AnalysisFolder + "Data/AccommodationDetails.pkl")
# AccommodationDetails.to_excel(AnalysisFolder + "Data/AccommodationDetails.xlsx", index = True)

In [ ]:
# Temporary import of the data (to be disabled when the notebook is fully operational)
# AccommodationDetails = pd.read_pickle(AnalysisFolder + "Data/AccommodationDetails.pkl")

In [ ]:
# Merging the data back into the main dataframe
Accommodations.drop(columns = ["Floor"], inplace = True) # to prevent duplicate cols
Accommodations = pd.merge(Accommodations, AccommodationDetails, how = "left", on = "Link")

### Extracting information on the landlord

Below, we extract information related to the landlord's trustworthiness, including how many accommodation entries they've posted, when they created their profile and when they posted the specific accommodation entry.

#### Extracting the data

In [ ]:
# Creating a dataframe to store the characteristics
LandloardDetails = pd.DataFrame()

# Extracting accommodation characteristics
for soup, id in zip(AllSoups, AllSoupIDs):
  #print(id) # can be un-commented for troubleshooting purposes
  TempLord = ExtractLandlordDetails(soup, id, DivClassGeneral = "css-ubwy5d", DivClassCreated = "css-a70nv0")
  LandloardDetails = pd.concat([LandloardDetails, TempLord], ignore_index = True)

# Cleaning up in the dataframe and previewing the data
LandloardDetails.reset_index(inplace = True, drop = True)

Unfortunately, the data comes in a format that makes it difficult to use as the preview below indicates. Particularly, we need to have numerical or date-like values in the `LandlordLastActive` and `LandlordSince` columns.

#### Formatting the data

In [ ]:
"""
=========================================
Repairing the "LandlordLastActive" column
=========================================
"""

# Marking whether the difference is expressed in days, months or years
LandloardDetails["LandlordLastActive_Days"] = (LandloardDetails["LandlordLastActive"].str.contains("dag")) & (LandloardDetails["LandlordLastActive"] != "I dag")
LandloardDetails["LandlordLastActive_Months"] = LandloardDetails["LandlordLastActive"].str.contains("måned")
LandloardDetails["LandlordLastActive_Years"] = LandloardDetails["LandlordLastActive"].str.contains("år")
LandloardDetails["LandlordLastActive_Today"] = (LandloardDetails["LandlordLastActive"] == "I dag")

# Replacing the boolean values with numerical values for days, months or years
LandloardDetails["LandlordLastActive_Days"] = np.where(LandloardDetails["LandlordLastActive_Days"] == True, LandloardDetails["LandlordLastActive"], np.nan)
LandloardDetails["LandlordLastActive_Days"] = LandloardDetails["LandlordLastActive_Days"].str.extract('(\d+)')
LandloardDetails["LandlordLastActive_Days"] = pd.to_numeric(LandloardDetails["LandlordLastActive_Days"])

LandloardDetails["LandlordLastActive_Months"] = np.where(LandloardDetails["LandlordLastActive_Months"] == True, LandloardDetails["LandlordLastActive"], np.nan)
LandloardDetails["LandlordLastActive_Months"] = LandloardDetails["LandlordLastActive_Months"].str.extract('(\d+)')
LandloardDetails["LandlordLastActive_Months"] = pd.to_numeric(LandloardDetails["LandlordLastActive_Months"])

LandloardDetails["LandlordLastActive_Years"] = np.where(LandloardDetails["LandlordLastActive_Years"] == True, LandloardDetails["LandlordLastActive"], np.nan)
LandloardDetails["LandlordLastActive_Years"] = LandloardDetails["LandlordLastActive_Years"].str.extract('(\d+)')
LandloardDetails["LandlordLastActive_Years"] = pd.to_numeric(LandloardDetails["LandlordLastActive_Years"])

# Following that, we also convert all values in these columns to days
LandloardDetails["LandlordLastActiveBeforeXDays"] = np.where(LandloardDetails["LandlordLastActive_Days"].notna(), LandloardDetails["LandlordLastActive_Days"], np.where(LandloardDetails["LandlordLastActive_Months"].notna(), LandloardDetails["LandlordLastActive_Months"]*30.4375, np.where(LandloardDetails["LandlordLastActive_Years"].notna(), LandloardDetails["LandlordLastActive_Years"]*365.25, np.nan)))
LandloardDetails["LandlordLastActiveBeforeXDays"].fillna(0, inplace = True)

# Dropping superfluous columns
LandloardDetails.drop(columns = ["LandlordLastActive_Days", "LandlordLastActive_Months", "LandlordLastActive_Years"], inplace = True)

In [ ]:
"""
====================================
Repairing the "LandlordSince" column
====================================
"""

# Marking whether the difference is expressed in days, months or years
LandloardDetails["LandlordSince_Days"] = LandloardDetails["LandlordSince"].str.contains("dag")
LandloardDetails["LandlordSince_Months"] = LandloardDetails["LandlordSince"].str.contains("måned")
LandloardDetails["LandlordSince_Years"] = LandloardDetails["LandlordSince"].str.contains("år")

# Replacing the boolean values with numerical values for days, months or years
LandloardDetails["LandlordSince_Days"] = np.where(LandloardDetails["LandlordSince_Days"] == True, LandloardDetails["LandlordSince"], np.nan)
LandloardDetails["LandlordSince_Days"] = LandloardDetails["LandlordSince_Days"].str.extract('(\d+)')
LandloardDetails["LandlordSince_Days"] = pd.to_numeric(LandloardDetails["LandlordSince_Days"])

LandloardDetails["LandlordSince_Months"] = np.where(LandloardDetails["LandlordSince_Months"] == True, LandloardDetails["LandlordSince"], np.nan)
LandloardDetails["LandlordSince_Months"] = LandloardDetails["LandlordSince_Months"].str.extract('(\d+)')
LandloardDetails["LandlordSince_Months"] = pd.to_numeric(LandloardDetails["LandlordSince_Months"])

LandloardDetails["LandlordSince_Years"] = np.where(LandloardDetails["LandlordSince_Years"] == True, LandloardDetails["LandlordSince"], np.nan)
LandloardDetails["LandlordSince_Years"] = LandloardDetails["LandlordSince_Years"].str.extract('(\d+)')
LandloardDetails["LandlordSince_Years"] = pd.to_numeric(LandloardDetails["LandlordSince_Years"])

# Following that, we also convert all values in these columns to days
LandloardDetails["LandlordSinceXDays"] = np.where(LandloardDetails["LandlordSince_Days"].notna(), LandloardDetails["LandlordSince_Days"], np.where(LandloardDetails["LandlordSince_Months"].notna(), LandloardDetails["LandlordSince_Months"]*30.4375, np.where(LandloardDetails["LandlordSince_Years"].notna(), LandloardDetails["LandlordSince_Years"]*365.25, np.nan)))
LandloardDetails["LandlordSinceXDays"].fillna(0, inplace = True)

# Dropping superfluous columns
LandloardDetails.drop(columns = ["LandlordSince_Days", "LandlordSince_Months", "LandlordSince_Years"], inplace = True)

In [ ]:
# Temporary export of the data (to be disabled when the notebook is fully operational)
# LandloardDetails.to_pickle(AnalysisFolder + "Data/LandloardDetails.pkl")
# LandloardDetails.to_excel(AnalysisFolder + "Data/LandloardDetails.xlsx", index = True)

In [ ]:
# Temporary import of the data (to be disabled when the notebook is fully operational)
# LandloardDetails = pd.read_pickle(AnalysisFolder + "Data/LandloardDetails.pkl")

In [ ]:
# Merging the data back into the main dataframe
Accommodations = pd.merge(Accommodations, LandloardDetails, how = "left", on = "Link")

### Calculating additional columns

Below, we add some further calculated columns, mostly related to the rent/deposit when adjusted for the size of the accommodation.

In [6]:
# Temporary import of the data (to be disabled when the notebook is fully operational)
# Accommodations = pd.read_pickle(AnalysisFolder + "Data/Accommodations.pkl")

In [ ]:
# Adjustments for rent
Accommodations["MonthlyRentBySqM"] = Accommodations["MonthlyRent"]/Accommodations["SizeSquareMeters"]
Accommodations["MonthlyRentByNumberOfRooms"] = Accommodations["MonthlyRent"]/Accommodations["NumberOfRooms"]

# Adjustments for aconto
Accommodations["AcontoBySqM"] = Accommodations["Aconto"]/Accommodations["SizeSquareMeters"]
Accommodations["AcontoByNumberOfRooms"] = Accommodations["Aconto"]/Accommodations["NumberOfRooms"]

# Adjustments for monthly fixed payment
Accommodations["MonthlyFixedPayment"] = Accommodations["MonthlyRent"] + Accommodations["Aconto"].fillna(0)
Accommodations["MonthlyFixedPaymentBySqM"] = Accommodations["MonthlyFixedPayment"]/Accommodations["SizeSquareMeters"]
Accommodations["MonthlyFixedPaymentByNumberOfRooms"] = Accommodations["MonthlyFixedPayment"]/Accommodations["NumberOfRooms"]

# Adjustments for deposit
Accommodations["DepositBySqM"] = Accommodations["Deposit"]/Accommodations["SizeSquareMeters"]
Accommodations["DepositByNumberOfRooms"] = Accommodations["Deposit"]/Accommodations["NumberOfRooms"]

# Adjustments for prepaid rent
Accommodations["PrepaidRentBySqM"] = Accommodations["PrepaidRent"]/Accommodations["SizeSquareMeters"]
Accommodations["PrepaidRentByNumberOfRooms"] = Accommodations["PrepaidRent"]/Accommodations["NumberOfRooms"]

# Adjustments for moving in price
Accommodations["MovingInPriceBySqM"] = Accommodations["MovingInPrice"]/Accommodations["SizeSquareMeters"]
Accommodations["MovingInPriceByNumberOfRooms"] = Accommodations["MovingInPrice"]/Accommodations["NumberOfRooms"]

Below, we add some information on whether the accommodations are located in the city or in the suburbs. Locations which are considered to be a part of the city are printed out below:

In [ ]:
# Adding information on whether the accommodation is located centrally or not
CentralLocations = ["København", "Frederiksberg"]
Accommodations["LocationInTheCity"] =  Accommodations["Municipality"].isin(CentralLocations)
Accommodations["LocationInTheSuburbs"] =  (Accommodations["LocationInTheCity"] == False)
Accommodations["LocationType"] = np.where(Accommodations["LocationInTheCity"] == True, "City", "Suburb")

# Printing a list of what locations are included under "City" to the user
print(CentralLocations)

['København', 'Frederiksberg']


Finally, we also perform some other, minor data corrections.

In [ ]:
# Accommodations of the "Room" type should always have 1 room
Accommodations["NumberOfRooms"] = np.where(Accommodations["AccommodationType"] == "Room", 1, Accommodations["NumberOfRooms"])

## Data export and clean-up

### Exporting data

Below, we export the `Accommodations` dataframe to the project folder so that it becomes available for use in other notebooks, where the focus is more on data visualization & analysis.

In [8]:
# Exporting the data both to PKL and XLSX
Accommodations.to_pickle(AnalysisFolder + "Data/Accommodations.pkl")
Accommodations.to_excel(AnalysisFolder + "Data/Accommodations.xlsx", index = False)

### Disconnecting from Google Drive (if relevant)

As a final step (and this only applies in case Google Drive is used as the file storage medium), we disconnect our session from it.

In [ ]:
"""
==========================================================
Disconnecting from Google Drive storage (only if relevant)
==========================================================
"""

if FileStorageForUse == "Drive":
  drive.flush_and_unmount()
  print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
